In [1]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.utils.data import DataLoader, TensorDataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from transformers import BertTokenizer, BertModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report

from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from torch.nn.utils.rnn import pad_sequence

In [2]:
# read tsv file cleaned_new
df = pd.read_csv('../data/final_cleaned.tsv', sep='\t')
df = df[:1000]
df.head()


,reference,translation,similarity,lenght_diff,ref_tox,trn_tox,ref_len,trn_len
0,alkar flood psychic waste explain high level n...,alkar flood mental waste would explain high le...,0.785171,0.010309,0.014195,0.981983,8,9
1,get nasty,become disgust,0.749687,0.071429,0.065473,0.999039,2,2
2,well could spare life one,well spare life,0.919051,0.268293,0.213313,0.985068,5,3
3,ah monkey get snap,monkey wake,0.664333,0.309524,0.053362,0.994215,4,2
4,get order put,order kill,0.726639,0.181818,0.009402,0.999348,3,2


### Embeddings:

Convert your tokenized text into numerical representations using word embeddings. You can use pre-trained embeddings (Word2Vec, GloVe, BERT) to capture the semantic meaning of words.

Padding and Sequence Length:
Ensure that all input sequences (toxic text) and output sequences (non-toxic text) have the same length. You may need to pad shorter sequences with a special token (e.g., <PAD>) or truncate longer sequences to a fixed length.

In [3]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [4]:
texts = df['reference'].tolist() + df['translation'].tolist()
toxicity = df['ref_tox'].tolist() + df['trn_tox'].tolist()
toxicity = [1 if x > 0.35 else 0 for x in toxicity]
input_ids = []
attention_masks = []

In [5]:
for text in texts:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=64,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\SunagatullinAyaz\.conda\envs\pythonProject2\lib\site-packages\transformers\tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [6]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
toxicity = torch.tensor(toxicity)

In [7]:
dataset = TensorDataset(input_ids, attention_masks, toxicity)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

In [8]:
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [10]:
epochs = 100

for epoch in range(epochs):
    print('Epoch:', epoch + 1)
    model.train()
    total_loss = 0
    for batch in train_dl:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_dl)
    print("Average train loss: {}".format(avg_train_loss))

Epoch: 1
Average train loss: 0.6990800607204437
Epoch: 2
Average train loss: 0.6645559120178223
Epoch: 3
Average train loss: 0.5889290785789489
Epoch: 4
Average train loss: 0.49561797618865966
Epoch: 5
Average train loss: 0.39916676640510557
Epoch: 6
Average train loss: 0.31641487032175064
Epoch: 7
Average train loss: 0.24131288081407548
Epoch: 8
Average train loss: 0.21309284888207913
Epoch: 9
Average train loss: 0.15780247017741203
Epoch: 10
Average train loss: 0.14695286829024554
Epoch: 11
Average train loss: 0.1122309335693717
Epoch: 12
Average train loss: 0.10008319124579429
Epoch: 13



KeyboardInterrupt



In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in val_dl:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_dlat = label_ids.flatten()

        total += labels_dlat.size
        correct += np.sum(pred_flat == labels_dlat)

print('Accuracy: %d %%' % (100 * correct / total))

In [ ]:
def predict_toxicity(text, model):
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=64,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    pred_flat = np.argmax(logits, axis=1).flatten()
    return pred_flat[0]

In [ ]:
print(predict_toxicity('I love you'))
print(predict_toxicity('I hate you'))

In [ ]:
print(predict_toxicity('I hate you so much asshole'))

In [ ]:
print(predict_toxicity('I love you so much'))

In [ ]:
# save model
torch.save(model.state_dict(), '../models/classification_model_NEW.pt')

In [ ]:
# load saved model
model_my = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
model_my.load_state_dict(torch.load('../models/classification_model_NEW.pt'))

In [ ]:
def predict_toxicity(text, model):
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=64,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    pred_flat = np.argmax(logits, axis=1).flatten()
    return pred_flat[0]

In [ ]:
model_my.to(device)
predict_toxicity('I love you', model_my)

In [ ]:
import gc

gc.collect()